In [ ]:
import pandas as pd
from rich import print
import datetime
pd.set_option('display.max_rows', 62)
pd.set_option('display.max_columns', 10)
import time
def read_qty_list_fun():
    ipdf = pd.read_excel('./CALL.xlsx')
    ipcsv = ipdf[['price','qty']]
    ipcsv = ipcsv.dropna()
    symbol_ip = ipdf['instrument'][0]
    inst_token = int(ipdf['token'][0])
    order_type = ipdf['order_type'][0]
    exch_type = ipdf['exch_type'][0]
    
    auth = ipdf[['ac_name','ac_code','psw','pin']][0:3]
    
    return {'ipcsv':ipcsv,'symbol_ip':symbol_ip,'inst_token':inst_token,
            'ipdf':ipdf,'auth':auth,'exch_type':exch_type,
            'order_type':order_type}

rd = read_qty_list_fun()
ipcsv = rd['ipcsv']
ipdf = rd['ipdf']
ipcsv['price'] = round(0.0025 * round(ipcsv['price']/0.0025), 4)
symbol_ip = rd['symbol_ip']
inst_token = rd['inst_token']

order_type = rd['order_type']
auth = rd['auth']

# exch_type_ip = rd['exch_type']
# exchange_type=''
# if(exch_type_ip=='CDS'):
#     exchange_type = module.kite.EXCHANGE_CDS
# if(exch_type_ip=='NFO'):
#     exchange_type = module.kite.EXCHANGE_NFO

print('symbol_ip : '+str(symbol_ip))
print('inst_token : '+str(inst_token))
print('order_type : '+str(order_type))
print(auth)

In [ ]:
trading_confirmed_df_ce = pd.DataFrame({'confirmed':False},index=[0])
trading_confirmed_df_ce.to_csv('./trading_confirmed_df_ce.csv',index=False)
    
stop_trading_triggered_ce = pd.DataFrame({'confirmed':False},index=[0])
stop_trading_triggered_ce.to_csv('./stop_trading_triggered_ce.csv',index=False)

init_trading = False
trading_confirmed_df_ce

In [ ]:
ipcsv.to_csv('./pivot_table_ce.csv',index=False)
ipcsv

In [ ]:
######## # AUTO LOGIN-1 #########
account_name = auth['ac_name'][0]

login_str = 'login_'+account_name
import threading
import importlib
login_str = 'login_'+account_name
module = importlib.import_module(login_str, package=None)

print('Logging in with account : '+str(account_name))
module.retry_autologin()

exch_type_ip = rd['exch_type']
exchange_type=''
if(exch_type_ip=='CDS'):
    exchange_type = module.kite.EXCHANGE_CDS
if(exch_type_ip=='NFO'):
    exchange_type = module.kite.EXCHANGE_NFO


In [ ]:
def get_orders_df():
    while True:
        orders_ipp = module.kite.orders()
        orders_df = pd.DataFrame(orders_ipp).sort_values(by=['order_timestamp'], ascending=False)
        orders_df.to_csv('./orders_ce.csv',index=False)
        time.sleep(3)
    
def get_positions_df():
    while True:
        positions = module.kite.positions()
        positions_df = pd.DataFrame(positions['net'])
        positions_df.to_csv('./positions_ce.csv',index=False)
        time.sleep(3)

ps_thr = threading.Thread(target=get_positions_df, args=())
ps_thr.start()

ord_thr = threading.Thread(target=get_orders_df, args=())
ord_thr.start()

In [ ]:
def stop_trading():
    global buy_depth_df
    cancel_all_pending_orders()
    sell_price_limit = buy_depth_df['price'][0]-5
    placeneworder(-myquantity,sell_price_limit)
    
    stop_trading_triggered_ce = pd.DataFrame({'confirmed':True},index=[0])
    stop_trading_triggered_ce.to_csv('./stop_trading_triggered_ce.csv',index=False)
    
    exit()

In [ ]:
def cancel_all_pending_orders():
    global order_type,symbol_ip,order_type
    myorder = module.kite.orders()
    pending_orders = []
    for i in range(len(myorder)):
        if ((myorder[i]['status'] == 'OPEN' or myorder[i]['status'] == 'AMO REQ RECEIVED' )
                and myorder[i]['tradingsymbol'] == symbol_ip
                and myorder[i]['product'] == order_type):
            pending_orders.append(myorder[i])
            module.kite.cancel_order(myorder[i]['variety'],
                              myorder[i]['order_id'],
                              parent_order_id=None)

In [ ]:
myquantity = 0
last_myquantity = 0

def getquant():
    global myquantity,order_type,symbol_ip,last_myquantity
    allpos = module.kite.positions()['net']
    for i in range(len(allpos)):
        if(allpos[i]['tradingsymbol']==symbol_ip and  allpos[i]['product']== order_type):
            myquantity = allpos[i]['quantity']
            print('My Quantity : ' + str(allpos[i]['quantity']))
            
#     if(myquantity == 0 and last_myquantity != 0):
#         stop_trading()
            
    last_myquantity = myquantity
    return myquantity

def placeneworder(quantdiff,price_ip1):
    '''placeneworder(quantdiff,price_ip1)'''
    print('--------------------------------------')
    print('placeneworder : '+str(quantdiff)+' '+str(price_ip1))
    print('--------------------------------------')
    global order_type,exchange_type
    if(quantdiff>0):
        try:
            order_id= module.kite.place_order(tradingsymbol=symbol_ip,
                        exchange=exchange_type,
                        transaction_type=module.kite.TRANSACTION_TYPE_BUY,
                        quantity=abs(int(quantdiff)),
                        price=price_ip1,
                        order_type=module.kite.ORDER_TYPE_MARKET if price_ip1 == 0 else module.kite.ORDER_TYPE_LIMIT,
                        variety = module.kite.VARIETY_REGULAR,
                        product=module.kite.PRODUCT_MIS if order_type=='MIS' else module.kite.PRODUCT_NRML)
#                 getquant()
        except Exception as e:
#             getquant()
            print(e)
    if(quantdiff<0):
        try:
            order_id= module.kite.place_order(tradingsymbol=symbol_ip,
                        exchange=exchange_type,
                        transaction_type=module.kite.TRANSACTION_TYPE_SELL,
                        quantity=abs(int(quantdiff)),
                        price=price_ip1,
                        order_type=module.kite.ORDER_TYPE_MARKET if price_ip1 == 0 else module.kite.ORDER_TYPE_LIMIT,
                        variety = module.kite.VARIETY_REGULAR,
                        product=module.kite.PRODUCT_MIS if order_type=='MIS' else module.kite.PRODUCT_NRML)
#                 getquant()
        except Exception as e:
#             getquant()
            print(e)

In [ ]:
myquantity = getquant()
myquantity

In [ ]:
def set_limits(priceipp,qtyipp,myquantity1):
    '''set_limits(priceipp,qtyipp,myquantity1)'''
    global order_type,inst_token,symbol_ip,ipcsv
    
    selected_index = ipcsv.loc[ipcsv['qty'] == qtyipp].index[0]

    prev_indx = selected_index -1
    next_indx = selected_index +1

    selected_price = round(0.0025 * round(ipcsv['price'][selected_index]/0.0025), 4)
    selected_qty = int(ipcsv['qty'][selected_index] - myquantity1)  

    prev_price = round(0.0025 * round(ipcsv['price'][prev_indx]/0.0025), 4)
    prev_qty = int(ipcsv['qty'][prev_indx] - myquantity1) - selected_qty
    next_price = round(0.0025 * round(ipcsv['price'][next_indx]/0.0025), 4)
    next_qty =  int(ipcsv['qty'][next_indx] - myquantity1) - selected_qty 
    
    
    if(prev_qty==0):
        prev_indx = prev_indx -1
        prev_price = round(0.0025 * round(ipcsv['price'][prev_indx]/0.0025), 4)
        prev_qty = int(ipcsv['qty'][prev_indx] - myquantity1) - selected_qty
        next_price = round(0.0025 * round(ipcsv['price'][next_indx]/0.0025), 4)
        next_qty =  int(ipcsv['qty'][next_indx] - myquantity1) - selected_qty 
        
    if(next_qty==0):
        next_indx = next_indx +1
        prev_price = round(0.0025 * round(ipcsv['price'][prev_indx]/0.0025), 4)
        prev_qty = int(ipcsv['qty'][prev_indx] - myquantity1) - selected_qty
        next_price = round(0.0025 * round(ipcsv['price'][next_indx]/0.0025), 4)
        next_qty =  int(ipcsv['qty'][next_indx] - myquantity1) - selected_qty 
    
    
    

#     print('selected_price : '+str(selected_price))
#     print('selected_qty : '+str(selected_qty))
#     print('--------------------------------------')
#     print('prev_price : '+str(prev_price))
#     print('prev_qty : '+str(prev_qty))
#     print('next_price : '+str(next_price))
#     print('next_qty : '+str(next_qty))
    
    x = threading.Thread(target=placeneworder, args=(prev_qty,prev_price))
    x.start()
    y = threading.Thread(target=placeneworder, args=(next_qty,next_price))
    y.start()

In [ ]:
def set_initial_order(ltpip,myqty_ip):
    '''new_set_limit(ltpip,myqty_ip)'''
    
    global order_type,inst_token,symbol_ip,ipcsv
    
    value = ltpip
    zero_index = ipcsv.index[ipcsv['qty'] == 0][0]
    pivot_price = ipcsv['price'][zero_index]

    index1 = abs(ipcsv['price'] - value).idxmin()
    
    selected_index = index1

    prev_indx = selected_index -1
    next_indx = selected_index +1

    selected_price = round(0.0025 * round(ipcsv['price'][selected_index]/0.0025), 4)
    selected_qty = int(ipcsv['qty'][selected_index]) 
    
#     print('ltpip : '+str(ltpip))
#     print('selected_index : '+str(selected_index))
#     print('prev_indx : '+str(prev_indx))
#     print('next_indx : '+str(next_indx))    
#     print('selected_price : '+str(selected_price))
#     print('selected_qty : '+str(selected_qty))
    
    prev_qty = int(ipcsv['qty'][prev_indx])
    next_qty =  int(ipcsv['qty'][next_indx])
    
    upper_qty =  prev_qty - myqty_ip
    lower_qty =  next_qty - myqty_ip
    
    if(upper_qty==0):
        prev_indx = prev_indx -1
        selected_price = round(0.0025 * round(ipcsv['price'][selected_index]/0.0025), 4)
        selected_qty = int(ipcsv['qty'][selected_index]) 

        prev_qty = int(ipcsv['qty'][prev_indx])
        next_qty =  int(ipcsv['qty'][next_indx])

        upper_qty =  prev_qty - myqty_ip
        lower_qty =  next_qty - myqty_ip
        
    if(lower_qty==0):
        next_indx = next_indx +1
        selected_price = round(0.0025 * round(ipcsv['price'][selected_index]/0.0025), 4)
        selected_qty = int(ipcsv['qty'][selected_index]) 

        prev_qty = int(ipcsv['qty'][prev_indx])
        next_qty =  int(ipcsv['qty'][next_indx])

        upper_qty =  prev_qty - myqty_ip
        lower_qty =  next_qty - myqty_ip
    

    prev_price = round(0.0025 * round(ipcsv['price'][prev_indx]/0.0025), 4)
    next_price = round(0.0025 * round(ipcsv['price'][next_indx]/0.0025), 4)
    
    print('selected_price : '+str(selected_price))
    print('selected_qty : '+str(selected_qty))
    print('--------------------------------------')
    print('prev_price : '+str(prev_price))
    print('upper_qty : '+str(upper_qty))
    print('next_price : '+str(next_price))
    print('lower_qty : '+str(lower_qty))
    
    x = threading.Thread(target=placeneworder, args=(upper_qty,prev_price))
    x.start()
    y = threading.Thread(target=placeneworder, args=(lower_qty,next_price))
    y.start()

In [ ]:
trading_confirmed_df_ce = pd.DataFrame({'confirmed':True},index=[0])
trading_confirmed_df_ce.to_csv('./trading_confirmed_df_ce.csv',index=False)

In [ ]:
####### START TRADING ######

import logging
from kiteconnect import KiteTicker
logging.basicConfig(level=logging.DEBUG)
# kws = KiteTicker("w19o0chuo929jxkp", "eA5B5OJQNOtZ0OYihkBmYw7Ke3B9pmCC")

cancel_all_pending_orders()
getquant()

prog_started = True

def on_ticks(ws, ticks):
    global order_type,inst_token,symbol_ip,ipcsv,myquantity,prog_started,init_trading,buy_depth_df,sell_depth_df
    print('==========================')
    print( 'LTP : ' + str(ticks[0]['last_price']))
    
    trading_confirmed_df_peip = pd.read_csv('./trading_confirmed_df_pe.csv')
    trading_confirmed_df_ceip = pd.read_csv('./trading_confirmed_df_ce.csv')
    ce_confirm = trading_confirmed_df_ceip['confirmed'][0]
    pe_confirm = trading_confirmed_df_peip['confirmed'][0]
    if(ce_confirm and pe_confirm):
        init_trading = True
    
    lk = read_qty_list_fun()
    ipcsv = lk['ipcsv']
    symbol_ip = lk['symbol_ip']
    inst_token = lk['inst_token']
    order_type = lk['order_type']
    
    buy_depth_df = pd.DataFrame(ticks[0]['depth']['buy'])
    sell_depth_df = pd.DataFrame(ticks[0]['depth']['sell'])
    tick_df = pd.DataFrame(ticks[0])
    buy_depth_df.to_csv('./buy_depth_df_ce.csv',index=False)
    sell_depth_df.to_csv('./sell_depth_df_ce.csv',index=False)
    tick_df.to_csv('./tick_df_ce.csv',index=False)
    
    stop_trading_triggered_peip = pd.read_csv('./stop_trading_triggered_pe.csv')
    pe_stoppedip = stop_trading_triggered_peip['confirmed'][0]
    if(pe_stoppedip):
        stop_trading()
    
    if(prog_started == True and init_trading == True):
        prog_started = False
        set_initial_order(sell_depth_df['price'][0],myquantity)

def on_connect(ws, response):
    global inst_token
    ws.subscribe([inst_token])
    ws.set_mode(ws.MODE_FULL, [inst_token])

def on_close(ws, code, reason):
    ws.stop()

def on_error(ws, code, reason):
    logging.error("closed connection on error: {} {}".format(code, reason))

def on_noreconnect(ws):
    logging.error("Reconnecting the websocket failed")

def on_reconnect(ws, attempt_count):
    logging.debug("Reconnecting the websocket: {}".format(attempt_count))
    
def on_order_update(ws, data):
    global order_type,inst_token,symbol_ip,ipcsv,myquantity
#     if(data['status']=='REJECTED' and data['tradingsymbol']== symbol_ip):
#         stop_trading()
    
    if(data['status']=='COMPLETE' and data['tradingsymbol']== symbol_ip):
        myquantity2 = getquant()
        
#         if(myquantity2 == 0):
#             stop_trading()
        
        cancel_all_pending_orders()
        avg_price = round(0.0025 * round(data['average_price']/0.0025), 4)
        print('order complete at : '+str(avg_price)+' '+str(data['quantity'])+' '+str(myquantity2))
        set_limits(avg_price,myquantity2,myquantity2)
        
#     print('--------------------------------------')
#     print("order update: ", data)
#     print('--------------------------------------')

module.kws.on_error = on_error
module.kws.on_noreconnect = on_noreconnect
module.kws.on_reconnect = on_reconnect
module.kws.on_order_update = on_order_update
module.kws.on_ticks = on_ticks
module.kws.on_connect = on_connect
module.kws.on_close = on_close

module.kws.connect()

In [ ]:
# stop_trading()

In [ ]:
from IPython.lib.display import Audio
import numpy as np

framerate = 10000
play_time_seconds = 1

t = np.linspace(0, play_time_seconds, framerate*play_time_seconds)
audio_data = np.sin(2*np.pi*1500*t) + np.sin(2*np.pi*1500*t)
Audio(audio_data, rate=framerate, autoplay=True)